In [ ]:
from fins3666.defines import *
from fins3666.utils import format_market_data, current_fx_data
from fins3666.portfolio import Portfolio
from fins3666.models.carry_trade import generate_orders

startTime = datetime(2001, 1, 1)

dates = mkt.loc[mkt['Timestamp'] >= startTime, 'Timestamp']
endTime = dates.iloc[-1]


fx_start = pd.DataFrame(current_fx_data(fx, startTime))


startingAssets = [
    {'asset': 'USD', 'units': 0, 'unit_value_USD': 1.0}]

date = startTime
currentData = current_fx_data(fx, date)

SamplePortfolio = Portfolio(startTime, startingAssets, currentData)


print(SamplePortfolio)
account_reference = ACCOUNT_SIZE_USD
prev_month = startTime.month

bugger = 0

for date in dates:
    month = date.month
    yields = mkt[mkt['Timestamp'] <= date]

    fx_rates = current_fx_data(fx, date)

    SamplePortfolio.updateMarketData(
        currentData=fx_rates, yields=yields)

    # Loop that runs once per month
    if (month != prev_month):
        prev_month = month
        # determine which assets to long buy and which to short
        current_positions = SamplePortfolio.summary()
        fx_arrays = SamplePortfolio.forex_spreads

        newOrders = generate_orders(
            yields=yields,
            current_positions=SamplePortfolio.summary(),
            fx=fx_arrays,
            size=account_reference)

        for order in newOrders:
            SamplePortfolio.new_order(order)

        print(SamplePortfolio)
        account_reference = index_benchmark(ACCOUNT_SIZE_USD, date)

    SamplePortfolio.update(date)


print(SamplePortfolio)



                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00             0     $              0.00             Long           0.00%         nan%

Portfolio - 1 assets
Timestamp: 2001-01-01 00:00:00,
Net Value ($USD): $0.00


                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00             0     $              0.00             Long           0.00%         nan%

Portfolio - 1 assets
Timestamp: 2001-01-31 00:00:00,
Net Value ($USD): $0.00


                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00             0     $   